In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install keras
!pip install pytube
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 21.9 MB/s eta 0:00:00


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import os
import pandas as pd
from pytube import YouTube
import yt_dlp

In [ ]:
# Read CSV file, trying different encodings
csv_file_path = '/content/drive/MyDrive/NeurIPS/Code/dataset_links_drunk.csv'

try:
    df = pd.read_csv(csv_file_path, encoding='utf-8')  # Try UTF-8 first
except UnicodeDecodeError:
    try:
        df = pd.read_csv(csv_file_path, encoding='latin-1')  # Try Latin-1 if UTF-8 fails
    except UnicodeDecodeError:
        df = pd.read_csv(csv_file_path, encoding='cp1252')  # Try cp1252 as a last resort

# Extract video links
print(df.head())
video_links = df['link'].tolist()

                                                link                    name  \
0  https://www.youtube.com/watch?v=x24jb5ww4lQ&li...  videoplayback (40).mp4   
1        https://www.youtube.com/watch?v=bvfEYtzq3fE  videoplayback (39).mp4   
2        https://www.youtube.com/watch?v=4QdSIQonzI4  videoplayback (38).mp4   
3        https://www.youtube.com/watch?v=H1KluhZpT1Y  videoplayback (37).mp4   
4        https://www.youtube.com/watch?v=SIH90qFVT04  videoplayback (36).mp4   

  Unnamed: 2  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  


In [ ]:
def download_youtube_video(url, output_path):
    yt = YouTube(url)
    stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
    stream.download(output_path=output_path)

In [ ]:
videos_folder = '/content/videos'
if not os.path.exists(videos_folder):
    os.makedirs(videos_folder)

In [ ]:
for link in video_links:
    try:
        download_youtube_video(link, videos_folder)
    except Exception as e:
        print(f"Failed to download {link}: {e}")

Failed to download https://www.youtube.com/watch?v=bvfEYtzq3fE: bvfEYtzq3fE is age restricted, and can't be accessed without logging in.
Failed to download https://www.youtube.com/watch?v=4QdSIQonzI4: 4QdSIQonzI4 is age restricted, and can't be accessed without logging in.
Failed to download https://www.youtube.com/watch?v=lSQkEALRHWU: lSQkEALRHWU is unavailable
Failed to download https://www.youtube.com/watch?v=BzkRyn6YEkw: BzkRyn6YEkw is age restricted, and can't be accessed without logging in.
Failed to download https://www.youtube.com/watch?v=C6UWFtqfsRo: C6UWFtqfsRo is age restricted, and can't be accessed without logging in.


KeyboardInterrupt: 

In [ ]:
def extract_frames(video_path, output_folder, frame_rate=30):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            frame_name = os.path.join(output_folder, f'frame_{count}.jpg')
            cv2.imwrite(frame_name, frame)
        count += 1
    cap.release()

def detect_faces(image_path, face_cascade):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    return faces

def draw_bounding_boxes(image, faces):
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)  # Draw bounding box around the face
    return image

def crop_to_bounding_box(image, face):
    x, y, w, h = face
    return image[y:y+h, x:x+w]

# Load Haar cascades
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
import glob

dataset_folder = '/content/drive/MyDrive/NeurIPS/Code/dataset_links_drunk.csv'
def process_videos(dataset_folder):
    video_files = glob.glob(os.path.join(dataset_folder, '*.mp4'))
    for video_file in video_files:
        video_name = os.path.basename(video_file).split('.')[0]
        frames_folder = os.path.join(dataset_folder, video_name, 'frames')
        extract_frames(video_file, frames_folder)

        frame_files = glob.glob(os.path.join(frames_folder, '*.jpg'))
        for frame_file in frame_files:
            faces = detect_faces(frame_file, face_cascade)
            img = cv2.imread(frame_file)
            img_with_boxes = draw_bounding_boxes(img, faces)

            # Save the frame with bounding boxes
            frame_with_boxes_path = os.path.join(dataset_folder, video_name, 'frames_with_boxes', os.path.basename(frame_file))
            if not os.path.exists(os.path.dirname(frame_with_boxes_path)):
                os.makedirs(os.path.dirname(frame_with_boxes_path))
            cv2.imwrite(frame_with_boxes_path, img_with_boxes)

            # Crop and save the face images
            for i, face in enumerate(faces):
                cropped_face = crop_to_bounding_box(img, face)
                cropped_face_path = os.path.join(dataset_folder, video_name, 'cropped_faces', f'{os.path.basename(frame_file).split(".")[0]}_face_{i}.jpg')
                if not os.path.exists(os.path.dirname(cropped_face_path)):
                    os.makedirs(os.path.dirname(cropped_face_path))
                cv2.imwrite(cropped_face_path, cropped_face)

In [ ]:
process_videos(videos_folder)

In [ ]:
import shutil

source_root = '/content/svideos/'
destination_directory = '/content/drunk6789v _face_images'
os.makedirs(destination_directory, exist_ok=True)

image_count = 0

for root, dirs, files in os.walk(source_root):
    for dir_name in dirs:
        if dir_name == 'frames_with_boxes':
            frames_with_boxes_path = os.path.join(root, dir_name)
            for image_name in os.listdir(frames_with_boxes_path):
                image_path = os.path.join(frames_with_boxes_path, image_name)
                if os.path.isfile(image_path):
                    new_image_name = f'SoberFace{image_count}.jpg'
                    new_image_path = os.path.join(destination_directory, new_image_name)
                    shutil.copy(image_path, new_image_path)
                    image_count += 1

        if dir_name == 'cropped_faces':
            cropped_faces_path = os.path.join(root, dir_name)
            for image_name in os.listdir(cropped_faces_path):
                image_path = os.path.join(cropped_faces_path, image_name)
                if os.path.isfile(image_path):
                    new_image_name = f'CroppedFace{image_count}.jpg'
                    new_image_path = os.path.join(destination_directory, new_image_name)
                    shutil.copy(image_path, new_image_path)
                    image_count += 1

print(f"Total images copied: {image_count}")

Total images copied: 53348


In [ ]:
import os
import shutil
import random

def reduce_image_count(input_folder, output_folder, target_count=10000):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    all_images = glob.glob(os.path.join(input_folder, '*.jpg'))

    if len(all_images) <= target_count:
        print(f"Already less than or equal to {target_count} images.")
        return

    selected_images = random.sample(all_images, target_count)

    for image_path in selected_images:
        shutil.move(image_path, os.path.join(output_folder, os.path.basename(image_path)))

input_folder = '/content/sober_faces'
output_folder = '/content/sober_faces_reduced'

In [ ]:
reduce_image_count(input_folder, output_folder)

In [ ]:
import os
import glob

def count_images_in_folder(folder):
    image_files = glob.glob(os.path.join(folder, '*.jpg'))
    return len(image_files)

reduced_folder = '/content/drunk_faces_reduced'

num_images = count_images_in_folder(reduced_folder)
print(f"Number of images in the reduced folder: {num_images}")


Number of images in the reduced folder: 10000


In [ ]:
!zip -r /content/sober_faces_reduced.zip /content/sober_faces_reduced/

Streaming output truncated to the last 5000 lines.
  adding: content/sober_faces_reduced/CroppedFace22206.jpg (deflated 3%)
  adding: content/sober_faces_reduced/CroppedFace20291.jpg (deflated 5%)
  adding: content/sober_faces_reduced/CroppedFace41173.jpg (deflated 6%)
  adding: content/sober_faces_reduced/CroppedFace996.jpg (deflated 5%)
  adding: content/sober_faces_reduced/SoberFace46084.jpg (deflated 0%)
  adding: content/sober_faces_reduced/SoberFace4060.jpg (deflated 1%)
  adding: content/sober_faces_reduced/SoberFace45944.jpg (deflated 0%)
  adding: content/sober_faces_reduced/SoberFace19782.jpg (deflated 1%)
  adding: content/sober_faces_reduced/SoberFace21212.jpg (deflated 1%)
  adding: content/sober_faces_reduced/SoberFace47524.jpg (deflated 1%)
  adding: content/sober_faces_reduced/SoberFace4290.jpg (deflated 1%)
  adding: content/sober_faces_reduced/SoberFace29456.jpg (deflated 0%)
  adding: content/sober_faces_reduced/CroppedFace1740.jpg (deflated 5%)
  adding: content/sob

In [ ]:
import shutil

shutil.rmtree('/content/svideos')